In [2]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/amir/projects/frame_stack


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import join as jn
import os
import yaml
import torch
from torchinfo import summary
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset


In [4]:
from video_module import *

In [5]:
with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)
with open('pathes.yaml') as conf_file:
    path_config = yaml.safe_load(conf_file)

pressure_path = path_config['p_video_path']
signal_path = path_config['test_s_video_path']


KeyError: 'test_s_video_path'

In [8]:
v_ds = Video_dataset(pressure_path, signal_path)
v_ds.split_to_chains(2)

In [6]:
for x in v_ds:
    print(x[0].shape, x[1].shape)

(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)
(30, 4, 64) (30, 64, 64)


In [11]:
len(v_ds)/299

64.0

In [27]:
v_dl = DataLoader(v_ds, batch_size=5)
signal, pressure = next(iter(v_dl))
signal.shape

torch.Size([5, 30, 4, 64])

In [31]:
subDL = SubDataLoader(signal, initial_pressure=pressure[:, 0])


In [33]:
for i in range(1, len(subDL)):
    input = subDL.get_next_batch()
    subDL.set_previous_pressure(pressure[:, i])
    if i == 7:
        break

In [37]:
input[0].shape, input[1].shape, input[2].shape

(torch.Size([5, 64, 64]), torch.Size([5, 4, 64]), torch.Size([5, 4, 64]))

In [38]:
from models_src import *

In [39]:
model = TestModel((64, 64), (4, 64))
model(*input).shape

torch.Size([5, 64, 64])

In [47]:
signal.shape

torch.Size([5, 30, 4, 64])

In [46]:
predict_chain_batch(model, signal, initial_pressure=pressure[:, 0]).shape

torch.Size([5, 256, 64])

In [66]:
from video_module import *

In [60]:
optim = torch.optim.Adam(model.parameters(), lr=1e-4)
fit_epoch(model, v_ds, torch.nn.MSELoss(), optim, 30, 5, 'cpu')

torch.Size([5, 30, 4, 64])
30
0.23783138394355774
torch.Size([5, 30, 4, 64])
30
0.25069481134414673
torch.Size([5, 30, 4, 64])
30
0.23790596425533295
torch.Size([5, 30, 4, 64])
30
0.2516731917858124
torch.Size([5, 30, 4, 64])
30
0.23971502482891083
torch.Size([5, 30, 4, 64])
30
0.23799049854278564
torch.Size([5, 30, 4, 64])
30
2677.748046875
torch.Size([5, 30, 4, 64])
30
6886.7255859375
torch.Size([2, 30, 4, 64])
30
7402.98291015625


1491.3241199702024

In [49]:
torch.rand(10, 10)[:, None].shape

torch.Size([10, 1, 10])

In [58]:
signal.size(0)

5

In [63]:
signal[None].shape

torch.Size([1, 5, 30, 4, 64])

In [64]:
signal_np = np.load(jn(signal_path, '0.npy'))

In [68]:
signal_np.shape

(297, 4, 64)

In [79]:
predict(model, signal_np, 'cpu').shape

(296, 64, 64)

In [73]:
torch.tensor(signal_np)[None].shape

torch.Size([1, 297, 4, 64])

## Dynamic dataset slicing test

In [25]:
from video_module import *
ds = Dynamic_video_dataset(pressure_path, signal_path)
len(ds.pressure[1:10:4])

3

## hdf5 datasets

In [16]:
from video_module import *

In [14]:
train_dataset = Stack_dataset('data/video/pressure/train', 'data/video/signal/train', 1, 1)

Dataset loading:   0%|          | 0/1 [00:00<?, ?video/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

IndexError: list index out of range

In [7]:
import h5py

train_pressure_file = h5py.File('data/video/pressure/train/train_videos.hdf5', 'r')


In [25]:
%%timeit
train_pressure_file["100_0"][:]

4.06 ms ± 55.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
keys = list(train_pressure_file.keys())[::10]
res = []
with tqdm(total=len(keys)) as pbar:
    for key in keys:
        a = train_pressure_file[key][:].astype(np.float32)
        res.append(a)
        pbar.set_postfix(file=key, shape=a.shape)
        pbar.update()

  0%|          | 0/1114 [00:00<?, ?it/s]

In [8]:
train_signal_file = h5py.File('data/video/signal/train/train_videos.hdf5', 'r')
with tqdm(total=len(train_signal_file.keys())) as pbar:
    for key in train_signal_file.keys():
        a = train_signal_file[key][:]
        pbar.set_postfix(file=key, shape=a.shape)
        pbar.update()

  0%|          | 0/11136 [00:00<?, ?it/s]

In [18]:
import hdf5plugin

In [23]:
old_file = h5py.File('data/video/pressure/test/test.hdf5', 'r')
new_file = h5py.File('data/video/pressure/test/test16.hdf5', 'w')

In [24]:
for key in tqdm(old_file.keys()):
    new_file.create_dataset(key, data=old_file[key][:].astype(np.float16), **hdf5plugin.Blosc(cname='blosclz', clevel=4, shuffle=hdf5plugin.Blosc.SHUFFLE))
new_file.flush()

  0%|          | 0/1024 [00:00<?, ?it/s]

In [25]:
len(new_file.keys())

1024

In [22]:
old_file.close()
new_file.close()